# Model A

## Prepare train data

In [ ]:
#java -mx3g -cp "C:\Users\zigsi\Desktop\CoreNLP\stanford-corenlp-full-2018-10-05/*" edu.stanford.nlp.naturalli.OpenIE text.txt

In [1]:
f = open("data/Weightless_dataset_train_A.csv", "r", encoding="utf-8")
first = True
questions = []
answers = []
grades = []
texts = []
for line in f:
    if first:
        first = False
        continue
    s = line.split(";")
    questions.append(s[10])
    answers.append(s[11])
    grades.append(s[14])
    texts.append(s[15])
    
print(texts[0])

Just the thought of leaving Earth both thrilled and terrified her. Her heart stopped as the trailer-sized shuttle moved forward on the track without making a sound. She took in a deep breath. she said, trying to be brave



## Prepare test data (first question)

In [2]:
f = open("data/Weightless_dataset_train.csv", "r", encoding="utf8")

from nltk.sem import 

first = True
questions_all = []
answers_all = []
grades_all = []
texts_all = []
for line in f:
    if first:
        first = False
        continue
    s = line.split(";")
    questions_all.append(s[10])
    answers_all.append(s[11])
    grades_all.append(s[14])
    texts_all.append(s[15])
test_answers = [answers_all[i] for i in range(len(questions_all)) if questions_all[i] == questions[0]]
test_grades = [float(grades_all[i].replace(",", ".")) for i in range(len(questions_all)) if questions_all[i] == questions[0]]

## TF-IDF

## Preparation

In [59]:
import string
import nltk
from nltk.stem import WordNetLemmatizer
def getTokens(text):
    lowered = text.lower()
    table = text.maketrans({key: None for key in string.punctuation})
    lowered = lowered.translate(table)
    return nltk.word_tokenize(lowered)

def preporcess(text):
    tokens = getTokens(text)
    lemmatizer = WordNetLemmatizer()
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    return " ".join(lemmas)
    #pos_tag = nltk.pos_tag(lemmas)
    #print(pos_tag)
    #return " ".join([pt[0] for pt in pos_tag if pt[1] == "NN" or pt[1][0:2] == "VB" or pt[1] == "JJ"])


pre_answer = preporcess(answers[0])
pre_text = preporcess(texts[0])
#print(lemmas)

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [61]:
vect = TfidfVectorizer() # parameters for tokenization, stopwords can be passed
#tfidf = vect.fit_transform([texts[0], answers[0]])
tfidf = vect.fit_transform([pre_answer, pre_text])
cosine = (tfidf * tfidf.T).A
#print("Cosine similarity between the documents: \n{}".format(cosine))

## Predicting

In [62]:
#weights = vect.transform(test_answers)
weights = vect.transform([preporcess(a) for a in test_answers])
predict = tfidf * weights.T
correct = 0
TP = 0
TN = 0
FP = 0
FN = 0
for i in range(predict.shape[1]):
    p = 0
    prediction = max(predict[0,i], predict[1,i])
    if prediction > 0.3: #0.5
        p = 1
    elif prediction > 0.2: #0.3
        p = 0.5
    else:
        p = 0
    #print("Real grade: ", test_grades[i], "Predicted grade: ", p)
    if abs(p - test_grades[i]) <= 0:
        correct += 1
    if p == test_grades[i]:
        #correct += 1
        if p == 1 or p == 0.5:
            TP += 1
        else:
            TN += 1
    else:
        if p == 1 or p == 0.5:
            FP += 1
        else:
            FN += 1
print("Correct: ", correct, "/", predict.shape[1])

Correct:  44 / 81


[texts[0] + answers[0]] 0.3 0.2 -> 38/81 correct

[texts[0], answers[0]]  0.3 0.2 -> 42/81 correct 

## F1 Scoring

In [63]:
T = TP + TN
A = T + FP + FN
P = TP/(TP+FP)
R = TP/(TP+FN)
F1 = 2*P*R/(P+R)
print("Classification Accuracy: ", T, "/", A, " = ", T/A)
print("Precision: ", TP, "/", TP+FP, " = ", P)
print("Recall: ", TP, "/", TP+FN, " = ", TP/(TP+FN))
print("F1: ", 2*P*R, "/", P+R, " = ", F1)

Classification Accuracy:  44 / 81  =  0.5432098765432098
Precision:  43 / 73  =  0.589041095890411
Recall:  43 / 50  =  0.86
F1:  1.0131506849315068 / 1.449041095890411  =  0.6991869918699187
